In [1]:
from deltalake import DeltaTable
import prefect as pf
import prefect.blocks.system as pfbs


In [2]:
import s3fs

In [3]:
aws_access_key = (await pfbs.Secret.load("minio-access-key")).get()
aws_secret_key = (await pfbs.Secret.load("minio-secret-key")).get()


In [4]:

pl_s3_delta_config = {
    "endpoint": "http://localhost:9000",
    "access_key_id": aws_access_key,
    "secret_access_key": aws_secret_key,
    "region": "us-east-1",
    "allow_http": "true",
}

s3_fs = s3fs.S3FileSystem(
    key=aws_access_key, secret=aws_secret_key, client_kwargs={"endpoint_url": "http://localhost:9000"}
)


In [23]:
dt_aggs_silver = DeltaTable("s3://etl/polygon/silver/daily_aggs/", storage_options=pl_s3_delta_config)

In [25]:
import polars as pl

In [26]:
pl.scan_pyarrow_dataset(dt_aggs_silver.to_pyarrow_dataset()).select(pl.col("ticker").unique()).collect()

ticker
str
"""AAPL"""
"""BBVA"""
"""NVDA"""
"""MSFT"""
